# 파이썬 알고리즘 스터디 9장
# 스택, 큐

> **스택 :** LIFO(후입선출) - 잔뜩 쌓아둔 접시들과 같음. 마지막에 쌓은 접시부터 제일 먼저 꺼내게 됨.  
**큐 :** FIFO(선입선출) - 맛집에 입장하기 위해 줄을 서는 것과 같음. 가장 먼저 줄을 선 사람이 가장 먼 저 입장.

파이썬은 스택 자료형을 별도 제공 X  
하지만, 리스트가 스택과 큐의 모든 연산을 지원. 다만, 리스트는 동적 배열로 구성되어 있어 큐의 효율적인 연산을 수행하기 위해서는 데크라는 별도 자료형 사용. 성능 고려하지 않는다면 리스트를 잘 사용하기만 해도 충분

## 스택

## 1. 유효한 괄호

> Q. 괄호로 된 입력값이 올바른지 판별하라.
- Open brackets must be closed by the same type of brackets.
- Open brackets must be closed in the correct order.

- https://leetcode.com/problems/valid-parentheses/

In [63]:
input1_1 = '(){}[]'
input1_2 = '()'
input1_3 = '(}'
input1_4 = '{[]}'

### my sol

In [91]:
def my_parentheses(s):

    left = collections.deque()
    temp = {
             '(':')',
             '{':'}',
             '[':']'
            }

    for char in s:
        if char in temp:
            left.append(char)

        else:
            if not left or temp[left.pop()] != char: 
                # 왼쪽에 해당하는 괄호가 아닐 때, 가장 마지막 left 값이 현재 괄호와 짝이 아닐 시 False
                return False
    
    return True

In [93]:
print(my_parentheses(input1_1))
print(my_parentheses(input1_2))
print(my_parentheses(input1_3))
print(my_parentheses(input1_4))

True
True
False
True


### 풀이 9.1.1. 스택 일치 여부 판별

In [68]:
def isValid(s):
    stack = []
    table = {
             ')':'(',
             '}':'{',
             ']':'['
            }
    
    for char in s:
        if char not in table:
            stack.append(char)
            
        elif not stack or table[char] != stack.pop():
            return False
        
    return len(stack) == 0    

In [69]:
print(isValid(input1_1))
print(isValid(input1_2))
print(isValid(input1_3))
print(isValid(input1_4))

True
True
False
True


- 문제를 잘 이해하는 것이 가장 중요  
- 미리 짝을 지어놓고, 오른쪽 편 괄호가 올 때마다 처리하는 것이 해결법

## 2. 중복 문자 제거
- 난이도: *** (어려움)

> Q. 중복된 문자를 제거하고 사전식 순서로 나열하라.
- https://leetcode.com/problems/remove-duplicate-letters/

In [223]:
input2_1 = 'bcabc'
input2_2 = 'cbacdcbc'
input2_3 = "ecbacba"
input2_4 = 'bcadabc'

### my sol 1

In [207]:
def my_letters(s):
    temp = collections.deque(sorted(list(set(s))))
    result = []
    for i in range(len(s)):
        if s[i] == temp[0]:
            temp.popleft()
            result.append(s[i])

        elif (s[i] not in s[i+1:]) and (s[i] in temp):
            result.append(s[i])
            
    return ''.join(result)

In [209]:
print(my_letters(input2_1))
print(my_letters(input2_2))
print(my_letters(input2_3))
print(my_letters(input2_4))

# 몇 가지 시도 후 틀리는 예제를 input으로 두고 모든 경우에서 정답을 출력하도록 해보려 했지만 실패.

abc
adbc
eacb
adbc


- 실패...

### 풀이 9.2.1. 재귀를 이용한 분리

In [245]:
def removeLetters(s):
    unique = sorted(set(s))
    
    for i in unique:
        temp = s[s.index(i): ]
        if set(s) == set(temp):
            return i + removeLetters(temp.replace(i, '')) # temp에 아무 것도 남지 않았을 때, 백트래킹하여 최종 결과 출력
        
    return ''

In [247]:
print(removeLetters(input2_1))
print(removeLetters(input2_2))
print(removeLetters(input2_3))
print(removeLetters(input2_4))

abc
acdb
eacb
adbc


- 사전식 순서이므로 정렬 후 그 순서대로 진행을 해야 함.
- 글자 하나를 추가할 때마다 뒤의 중복 글자를 삭제
- 추가되는 글자와 그 뒤의 글자까지의 집합(set(temp))이 이미 추가된 글자를 제외한 집합(set(s))과 같아 함.
- 분리 진행하는 풀이법 익혀두자...
- 분리를 한 뒤에서 정렬 된 순서대로 진행하는 것이 포인트라고 생각됨.

### 풀이 9.2.2. 스택을 이용한 문자 제거

In [264]:
def removeLetters2(s):
    counter, seen, stack = collections.Counter(s), set(), []
    
    for char in s:
        counter[char] -= 1
        # 뒤에 해당 문자가 남아있는지의 확인을 in으로 처리하지 않았음!
        if char in seen:
            continue # seen은 이미 처리된 문자 집합. 즉, 이미 처리된 문자이면 스킵한다는 의미
            
        while stack and char < stack[-1] and counter[stack[-1]] > 0:
            # 1) stack에 문자가 있을 때
            # 2) 현재 문자가 이전 문자보다 앞서고
            # 3) 이전 문자가 아직 쌓을 문자열 중에 남아있으면
            seen.remove(stack.pop())
            # 4) 이전 문자 stack과 seen에서 삭제하고 위의 1), 2), 3)을 반복.
            
        stack.append(char)
        seen.add(char)
    
    return ''.join(stack)

- 어떻게 이런 생각을...

In [ ]:
input2_1 = 'bcabc'
input2_2 = 'cbacdcbc'
input2_3 = "ecbacba"
input2_4 = 'bcadabc'

In [258]:
collections.Counter(input2_1)

Counter({'b': 2, 'c': 2, 'a': 1})

In [259]:
'a' < 'b' # 문자열끼리도 크기 비교가 가능했음!

True

In [265]:
print(removeLetters2(input2_1))
print(removeLetters2(input2_2))
print(removeLetters2(input2_3))
print(removeLetters2(input2_4))

abc
acdb
eacb
adbc


## 3. 일일 온도

> Q. 매일의 화씨 온도 리스트 T를 입력 받아서, 더 따뜻한 날씨를 위해서는 며칠을 더 기다려야하는지를 출력하라.
- https://leetcode.com/problems/daily-temperatures/

In [330]:
T1 = [73, 74, 75, 71, 69, 72, 76, 73]
# [1, 1, 4, 2, 1, 1, 0, 0]
T2 = [55,38,53,81,61,93,97,32,43,78]
# [3,1,1,2,1,1,0,1,1,0]
T3 = [89,62,70,58,47,47,46,76,100,70]

### my sol 1 - time limit exceeded

In [334]:
def my_warmer(T):

    output = []
    for i in range(len(T)-1):
        left, right = i, i+1
        step = 1

        while (right < len(T)-1) and T[right] <= T[left]:
            right += 1

        if T[left] < T[right]:
            output.append(right - left)

        else:
            output.append(0)

    output.append(0)
    
    return output

In [336]:
print(my_warmer(T1))
print(my_warmer(T2))
print(my_warmer(T3))

[1, 1, 4, 2, 1, 1, 0, 0]
[3, 1, 1, 2, 1, 1, 0, 1, 1, 0]
[8, 1, 5, 4, 3, 2, 1, 1, 0, 0]


### 풀이 9.3.1. 스택 값 비교

In [337]:
def dailyTemperatures(T):
    answer = [0]*len(T)
    stack = []
    
    for i, cur in enumerate(T):
        # 현재 온도가 이전 스택 값보다 높다면 정답 처리
        while stack and cur > T[stack[-1]]:
            last = stack.pop()
            answer[last] = i - last
            
        stack.append(i)
        
    return answer

In [338]:
print(dailyTemperatures(T1))
print(dailyTemperatures(T2))
print(dailyTemperatures(T3))

[1, 1, 4, 2, 1, 1, 0, 0]
[3, 1, 1, 2, 1, 1, 0, 1, 1, 0]
[8, 1, 5, 4, 3, 2, 1, 1, 0, 0]


- 후입된 것부터 처리해나가는 스택 문제의 특성을 잘 기억하자.